# Lab | Random Forests

### Instructions
- Apply the Random Forests algorithm but this time only by upscaling the data to deal with the imbalance.
- Use Feature Selections that you have learned in class to decide if you want to use all of the features (Variance Threshold, RFE, PCA, etc.)
- Re-run the Random Forest algorithm to determine if the Feature Selection has improved the results.
- Discuss the output and its impact in the business scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the business?

In [1]:
import pandas as pd
import numpy as np

numerical   = pd.read_csv('./files_for_lab/numerical.csv')
categorical = pd.read_csv('./files_for_lab/categorical.csv')
target      = pd.read_csv('./files_for_lab/target.csv')
target

,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0
...,...,...
95407,0,0.0
95408,0,0.0
95409,0,0.0
95410,1,18.0


In [2]:
target['TARGET_B'].value_counts()

0    90569
1     4843
Name: TARGET_B, dtype: int64

In [3]:
display(numerical.shape)
categorical.shape

(95412, 315)

(95412, 22)

In [4]:
numerical.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
3,0,70.000000,1,4,2,0,23,14,31,3,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
4,0,78.000000,3,2,60,1,28,9,53,26,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26


In [5]:
categorical.head()

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,...,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,...,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,...,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,...,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,...,20,1,93,10,96,1,96,1,79,3


In [6]:
categorical.dtypes

STATE           object
CLUSTER          int64
HOMEOWNR        object
GENDER          object
DATASRCE         int64
RFA_2R          object
RFA_2A          object
GEOCODE2        object
DOMAIN_A        object
DOMAIN_B         int64
ODATEW_YR        int64
ODATEW_MM        int64
DOB_YR           int64
DOB_MM           int64
MINRDATE_YR      int64
MINRDATE_MM      int64
MAXRDATE_YR      int64
MAXRDATE_MM      int64
LASTDATE_YR      int64
LASTDATE_MM      int64
FIRSTDATE_YR     int64
FIRSTDATE_MM     int64
dtype: object

For simplicity, I will OneHot Encode only CLUSTER and DATASRCE.
All other categorical fields with integer type will be ordinally encoded by scaling with numericals.

In [7]:
categorical['CLUSTER'] = categorical['CLUSTER'].astype(object)
categorical['DATASRCE'] = categorical['DATASRCE'].astype(object)

In [8]:
numerical.isna().sum().sum()

0

In [9]:
categorical.isna().sum().sum()

0

So data is clean, but we have not chosen features or scaled/encoded the data.

In [10]:
y = target['TARGET_B']
X = pd.concat([numerical, categorical], axis=1)

### Upscaling, encoding and scaling data

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 123)

In [12]:
train = pd.concat([X_train, y_train], axis=1)

train_0 = train[train['TARGET_B']==0]
train_1 = train[train['TARGET_B']==1]

display(train_0.shape)
train_1.shape              # heavily imbalanced dataset

(72452, 338)

(3877, 338)

In [13]:
from sklearn.utils import resample

train_1_oversampled = resample(train_1, replace=True, n_samples = len(train_0))
display(train_0.shape)
train_1_oversampled.shape

(72452, 338)

(72452, 338)

In [14]:
train_upsampled = pd.concat([train_0, train_1_oversampled], axis=0)

# and to randomise the rows (although I will do this indirectly through encoding/scaling later):

train_upsampled = train_upsampled.sample(frac=1)

display(train_upsampled.shape)
train_upsampled.head()

# note the TARGET_B column on right is not ordered

(144904, 338)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B
22804,0,67.000000,5,9,0,0,24,38,36,1,...,12,89,9,95,4,96,1,87,3,0
50871,0,60.000000,6,6,3,0,24,31,69,14,...,5,89,8,95,11,95,12,87,1,1
59533,1,60.000000,3,9,0,1,28,35,15,6,...,1,94,1,95,12,95,12,94,1,0
31035,0,79.000000,5,9,0,0,48,27,40,22,...,1,92,6,96,3,96,3,91,10,0
7248,2,61.611649,1,9,0,0,30,39,37,8,...,2,90,2,95,12,95,12,86,12,1


In [15]:
# I will now reassign X_train and y_train to be based off this upsampled dataset, and then 

X_train = train_upsampled.drop('TARGET_B', axis=1)
y_train = train_upsampled['TARGET_B']

display(X_train.head())
y_train.head()

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
22804,0,67.000000,5,9,0,0,24,38,36,1,...,30,12,89,9,95,4,96,1,87,3
50871,0,60.000000,6,6,3,0,24,31,69,14,...,38,5,89,8,95,11,95,12,87,1
59533,1,60.000000,3,9,0,1,28,35,15,6,...,38,1,94,1,95,12,95,12,94,1
31035,0,79.000000,5,9,0,0,48,27,40,22,...,19,1,92,6,96,3,96,3,91,10
7248,2,61.611649,1,9,0,0,30,39,37,8,...,0,2,90,2,95,12,95,12,86,12


22804    0
50871    1
59533    0
31035    0
7248     1
Name: TARGET_B, dtype: int64

In [16]:
X_train_num = X_train.select_dtypes(np.number).copy()
X_test_num = X_test.select_dtypes(np.number).copy()
X_train_cat = X_train.select_dtypes(object).copy()
X_test_cat = X_test.select_dtypes(object).copy()

display(X_train_cat.shape) # expect only 9 (not 22) columns now as we will ordinally encode the integer categorical columns with numericals
display(X_train_cat.head())
y_train.head()

(144904, 9)

,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A
22804,MI,51,H,M,3,L,E,D,R
50871,other,40,H,M,1,L,F,C,T
59533,other,39,U,M,2,L,F,B,T
31035,CA,3,H,F,3,L,F,A,U
7248,other,49,H,M,2,L,G,D,R


22804    0
50871    1
59533    0
31035    0
7248     1
Name: TARGET_B, dtype: int64

The numericals do not need to be scaled for Decision Tree / Random Forest models, but for feature selection later on it will be easier if the dataset has been scaled.

In [17]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X_train_num) # fit only on the training set, but applied to both train and test (below)

X_train_scaled = pd.DataFrame(transformer.transform(X_train_num), columns=X_train_num.columns)
X_test_scaled = pd.DataFrame(transformer.transform(X_test_num), columns=X_train_num.columns)

display(X_train_scaled.describe().T)
X_test_scaled.describe().T

,count,mean,std,min,25%,50%,75%,max
TCODE,144904.0,0.000866,0.014603,0.0,0.000000,0.000014,0.000028,1.0
AGE,144904.0,0.627430,0.145697,0.0,0.536082,0.624862,0.731959,1.0
INCOME,144904.0,0.530999,0.281532,0.0,0.333333,0.666667,0.666667,1.0
WEALTH1,144904.0,0.786798,0.297611,0.0,0.666667,1.000000,1.000000,1.0
HIT,144904.0,0.014054,0.035641,0.0,0.000000,0.000000,0.012448,1.0
...,...,...,...,...,...,...,...,...
MAXRDATE_MM,144904.0,0.538334,0.362909,0.0,0.181818,0.545455,0.909091,1.0
LASTDATE_YR,144904.0,0.229369,0.271674,0.0,0.000000,0.000000,0.500000,1.0
LASTDATE_MM,144904.0,0.501063,0.389547,0.0,0.090909,0.454545,0.909091,1.0
FIRSTDATE_YR,144904.0,0.948651,0.033555,0.0,0.916667,0.947917,0.979167,1.0


,count,mean,std,min,25%,50%,75%,max
TCODE,19083.0,0.000725,0.012556,0.000000,0.000000,0.000014,0.000028,0.541679
AGE,19083.0,0.624277,0.150293,0.000000,0.525773,0.624862,0.731959,1.000000
INCOME,19083.0,0.523441,0.283671,0.000000,0.333333,0.666667,0.666667,1.000000
WEALTH1,19083.0,0.783891,0.301476,0.000000,0.555556,1.000000,1.000000,1.000000
HIT,19083.0,0.014330,0.043738,0.000000,0.000000,0.000000,0.012448,1.000000
...,...,...,...,...,...,...,...,...
MAXRDATE_MM,19083.0,0.538595,0.362678,0.000000,0.181818,0.545455,0.909091,1.000000
LASTDATE_YR,19083.0,0.209742,0.263734,0.000000,0.000000,0.000000,0.500000,1.000000
LASTDATE_MM,19083.0,0.513003,0.386048,0.000000,0.090909,0.545455,0.909091,1.000000
FIRSTDATE_YR,19083.0,0.951069,0.033097,0.802083,0.916667,0.958333,0.979167,1.000000


In [18]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first').fit(X_train_cat) # again, fit only on the training set, but applied to both
cols = encoder.get_feature_names_out(input_features=X_train_cat.columns)

X_train_encode = pd.DataFrame(encoder.transform(X_train_cat).toarray(),columns=cols)
X_test_encode = pd.DataFrame(encoder.transform(X_test_cat).toarray(),columns=cols)

display(X_train_encode.head())
X_test_encode.head()

,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,...,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


,STATE_FL,STATE_GA,STATE_IL,STATE_IN,STATE_MI,STATE_MO,STATE_NC,STATE_TX,STATE_WA,STATE_WI,...,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [19]:
display(y_train.head())
y_test.head()

22804    0
50871    1
59533    0
31035    0
7248     1
Name: TARGET_B, dtype: int64

10396    0
77799    0
66629    0
62633    0
83026    0
Name: TARGET_B, dtype: int64

In [20]:
X_train_transformed = pd.concat([X_train_scaled,X_train_encode], axis = 1)
X_test_transformed = pd.concat([X_test_scaled,X_test_encode], axis = 1)

y_train = y_train.reset_index(drop=True) # as row index / labels are no longer aligned with transformed X
y_test = y_test.reset_index(drop=True)

In [21]:
display(y_train.head())
y_test.head()

0    0
1    1
2    0
3    0
4    1
Name: TARGET_B, dtype: int64

0    0
1    0
2    0
3    0
4    0
Name: TARGET_B, dtype: int64

In [22]:
y_train.value_counts() # equal as we upsampled the train_1 data

0    72452
1    72452
Name: TARGET_B, dtype: int64

So entire dataframe is prepared for modelling. No features have been removed at this stage.

### First Random Forest Algorithm

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

# for simplicity, I will keep the parameters used in class

RFC = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=1, # fraction of X-train to use in each tree
                             random_state=42)

RFC.fit(X_train_transformed, y_train)

RFC_score = RFC.score(X_test_transformed, y_test) # I want this for comparison later

print('Training score: ', RFC.score(X_train_transformed, y_train))
print('Test score: ', RFC_score)

# to visualise the model output

y_pred = RFC.predict(X_test_transformed)
display(confusion_matrix(y_test, y_pred))

Training score:  0.5
Test score:  0.050620971545354505


array([[    0, 18117],
       [    0,   966]], dtype=int64)

In [24]:
y_test

0        0
1        0
2        0
3        0
4        0
        ..
19078    0
19079    0
19080    0
19081    1
19082    0
Name: TARGET_B, Length: 19083, dtype: int64

In [29]:
y_pred

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
